![Seneca College Logo](Seneca_College_Logo.png "Seneca College Logo")
<h1 style="text-align: center;"> Programming For Analytics Project </h1>
<p style="text-align: center;">Adventureworks - Territory Sales Project</p>
<p style="text-align: center;">By Tareq Haboukh</p>

![Phase 1](#Phase 1)

<a id='Phase 1'></a>
<h3>Phase 1: Data Import</h3>
<p>First, I assigned the library “project’ to the directory “~” and created a file reference using the
filename statement. Then to import the AdventureWorks.xlsx file I used the proc import step for
each sheet in the file and created a corresponding dataset.
Both Product and SalesOrderDetails sheets were not loaded because they are not being used.</p>

In [1]:
libname project '~';
filename reffile 'AdventureWorks.xlsx';
* 1. Data Import;
* Import SalesOrderHeader sheet;
proc import
	datafile=reffile
	dbms=xlsx
	out=project.SalesOrderHeader;
	sheet='SalesOrderHeader';
	getnames=yes;
run;
* Import SalesTerritory sheet;
proc import
	datafile=reffile
	dbms=xlsx
	out=project.SalesTerritory;
	sheet='SalesTerritory';
	getnames=yes;
run;

proc print data=project.SalesOrderHeader (obs=5); run;
proc print data=project.SalesTerritory (obs=5); run;

Obs,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
1,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279,5,985,985,5,16281,105041Vi84182,,20565.6206,1971.5149,616.0984,23153.2339,,{79B65321-39CA-4115-9CBA-8FE0903E12E6},2011-06-07 00:00:00
2,43660,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43660,PO18850127500,10-4020-000117,29672,279,5,921,921,5,5618,115213Vi29411,,1294.2529,124.2483,38.8276,1457.3288,,{738DC42D-D03B-48A1-9822-F95A67EA7389},2011-06-07 00:00:00
3,43661,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43661,PO18473189620,10-4020-000442,29734,282,6,517,517,5,1346,85274Vi6854,4,32726.4786,3153.7696,985.553,36865.8012,,{D91B9131-18A4-4A11-BC3A-90B6F53E9D74},2011-06-07 00:00:00
4,43662,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43662,PO18444174044,10-4020-000227,29994,282,6,482,482,5,10456,125295Vi53935,4,28832.5289,2775.1646,867.2389,32474.9324,,{4A1ECFC0-CC3A-4740-B028-1C50BB48711C},2011-06-07 00:00:00
5,43663,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43663,PO18009186470,10-4020-000510,29565,276,4,1073,1073,5,4322,45303Vi22691,,419.4589,40.2681,12.5838,472.3108,,{9B1E7A40-6AE0-4AD3-811C-A64951857C4B},2011-06-07 00:00:00
6,43664,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43664,PO16617121983,10-4020-000397,29898,280,1,876,876,5,806,95555Vi4081,,24432.6088,2344.9921,732.81,27510.4109,,{22A8A5DA-8C22-42AD-9241-839489B6EF0D},2011-06-07 00:00:00
7,43665,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43665,PO16588191572,10-4020-000146,29580,283,1,849,849,5,15232,35568Vi78804,,14352.7713,1375.9427,429.9821,16158.6961,,{5602C304-853C-43D7-9E79-76E320D476CF},2011-06-07 00:00:00
8,43666,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43666,PO16008173883,10-4020-000511,30052,276,4,1074,1074,5,13349,105623Vi69217,,5056.4896,486.3747,151.9921,5694.8564,,{E2A90057-1366-4487-8A7E-8085845FF770},2011-06-07 00:00:00
9,43667,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43667,PO15428132599,10-4020-000646,29974,277,3,629,629,5,10370,55680Vi53503,,6107.082,586.1203,183.1626,6876.3649,,{86D5237D-432D-4B21-8ABC-671942F5789D},2011-06-07 00:00:00
10,43668,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43668,PO14732180295,10-4020-000514,29614,282,6,529,529,5,1566,85817Vi8045,4,35944.1562,3461.7654,1081.8017,40487.7233,,{281CC355-D538-494E-9B44-461B36A826C6},2011-06-07 00:00:00


<h3>Phase 2: Data Cleaning</h3>
<h4>Creating the SalesOrderHeader_Clean</h4>

<p>First, I started with cleaning the SalesOrderHeader dataset and created a new one called
SalesOrderHeader_Clean by using a data step.
In this step, only SalesOrderID TotalDue OnlineOrderFlag OrderDate TerritoryID variables were
added by using the “keep” statement within the “set” statement, then I created new variables for
the ones I wanted to change from character to numeric and used “input” with the format needed
and named them by adding Num or Date to the original variable name.</p>

In [5]:
* 2. Data Cleaning;
* SalesOrderHeader_Clean;
data project.SalesOrderHeader_Clean;
	set 	project.SalesOrderHeader(keep=SalesOrderID TotalDue OnlineOrderFlag OrderDate TerritoryID);
			NumTotalDue=input(TotalDue,dollar16.2);
			NumOnlineOrderFlag=input(OnlineOrderFlag,8.);
			DateOrderDate=input(OrderDate,anydtdte10.);
			NumTerritoryID=input(TerritoryID,8.);
	format	NumTotalDue dollar16.2
			DateOrderDate mmddyy10.;
	drop	TotalDue
			OnlineOrderFlag
			OrderDate
			TerritoryID;
	rename	NumTotalDue=TotalDue
			NumOnlineOrderFlag=OnlineOrderFlag
			DateOrderDate=OrderDate
			NumTerritoryID=TerritoryID;
run;

proc print data=project.SalesOrderHeader_Clean (obs=10); run;

Obs,SalesOrderID,TotalDue,OnlineOrderFlag,OrderDate,TerritoryID
1,43659,"$23,153.23",0,05/31/2011,5
2,43660,"$1,457.33",0,05/31/2011,5
3,43661,"$36,865.80",0,05/31/2011,6
4,43662,"$32,474.93",0,05/31/2011,6
5,43663,$472.31,0,05/31/2011,4
6,43664,"$27,510.41",0,05/31/2011,1
7,43665,"$16,158.70",0,05/31/2011,1
8,43666,"$5,694.86",0,05/31/2011,4
9,43667,"$6,876.36",0,05/31/2011,3
10,43668,"$40,487.72",0,05/31/2011,6


<h4>Creating the Territory_Clean dataset</h4>

<p>The same was done to the Territory dataset and the Territory_Clean dataset was created. Kept only
TerritoryID Name CountryRegionCode Group SalesYTD variables and changed SalesYTD and
TerritoryID from character to numeric and applied the format needed.</p>

In [4]:
* Territory_Clean;
data project.Territory_Clean;
	set 	project.SalesTerritory(keep=TerritoryID Name CountryRegionCode Group SalesYTD);
			NumSalesYTD=input(SalesYTD,dollar16.2);
			NumTerritoryID=input(TerritoryID,8.);
	format	NumSalesYTD dollar16.2
			NumTerritoryID 8.;
	drop	SalesYTD TerritoryID;
	rename	NumSalesYTD=SalesYTD NumTerritoryID=TerritoryID;
run;

proc print data=project.Territory_clean (obs=10); run;

Obs,Name,CountryRegionCode,Group,SalesYTD,TerritoryID
1,Northwest,US,North America,"$7,887,186.79",1
2,Northeast,US,North America,"$2,402,176.85",2
3,Central,US,North America,"$3,072,175.12",3
4,Southwest,US,North America,"$10,510,853.87",4
5,Southeast,US,North America,"$2,538,667.25",5
6,Canada,CA,North America,"$6,771,829.14",6
7,France,FR,Europe,"$4,772,398.31",7
8,Germany,DE,Europe,"$3,805,202.35",8
9,Australia,AU,Pacific,"$5,977,814.92",9
10,United Kingdom,GB,Europe,"$5,012,905.37",10


<h3>Phase 3: Joining and Merging</h3>
<h4>Creating the SalesDetails dataset</h4>

<p>To join the two new datasets, we first need to sort them by the TerritoryID variable using the proc
sort step and then a data merge step that creates a new dataset called SalesDetails by matching
records from SalesOrderHeader_Clean with values from Territory_Clean if they have the same
TerritoryID in a one-to-many join.</p>

In [5]:
* Joining & Merging;
proc sort
	data=project.SalesOrderHeader_Clean;
	by TerritoryID;
run;
proc sort
	data=project.territory_clean;
	by TerritoryID;
run;
data project.SalesDetails;
	merge	project.SalesOrderHeader_Clean(in=Q1)
			project.territory_clean(in=Q2);
	by		TerritoryID;
	if 		Q1 = 1 and Q2 =1;
run;

proc print data=project.SalesDetails(obs=10); run;

Obs,SalesOrderID,TotalDue,OnlineOrderFlag,OrderDate,TerritoryID,Name,CountryRegionCode,Group,SalesYTD
1,43664,"$27,510.41",0,05/31/2011,1,Northwest,US,North America,"$7,887,186.79"
2,43665,"$16,158.70",0,05/31/2011,1,Northwest,US,North America,"$7,887,186.79"
3,43669,$807.26,0,05/31/2011,1,Northwest,US,North America,"$7,887,186.79"
4,43671,"$9,153.61",0,05/31/2011,1,Northwest,US,North America,"$7,887,186.79"
5,43683,"$48,204.07",0,05/31/2011,1,Northwest,US,North America,"$7,887,186.79"
6,43686,"$3,899.68",0,05/31/2011,1,Northwest,US,North America,"$7,887,186.79"
7,43699,"$3,756.99",-1,05/31/2011,1,Northwest,US,North America,"$7,887,186.79"
8,43711,"$3,953.99",-1,06/03/2011,1,Northwest,US,North America,"$7,887,186.79"
9,43718,"$3,953.99",-1,06/05/2011,1,Northwest,US,North America,"$7,887,186.79"
10,43723,$772.50,-1,06/06/2011,1,Northwest,US,North America,"$7,887,186.79"


<h4>Creating the SalesAnalysis dataset</h4>

<p>To create the SalesAnalysis dataset I used the “Obtaining a total for each by group in SAS” method
using the following code, this method consolidates all rows with the same TerritoryID and sums
the TotalDue into a new variable SubtTotal.and creates the SalesAnalysis dataset<p>

In [6]:
* Sales Analysis;
proc sort
	data=project.SalesDetails
	out=project.SortedSalesDetails;
	by TerritoryID;
run;
data project.SalesAnalysis(drop=TotalDue TerritoryID SalesOrderID OnlineOrderFlag OrderDate);
	retain	Name CountryRegionCode Group SalesYTD SubTotal;
	format	SubTotal dollar16.2;
	set		project.SortedSalesDetails;
	by		TerritoryID;
	if		First.TerritoryID then SubTotal=0;
			SubTotal + TotalDue;
	if		Last.TerritoryID;   
run;

proc print data=project.SalesAnalysis noobs; run;

Name,CountryRegionCode,Group,SalesYTD,SubTotal
Northwest,US,North America,"$7,887,186.79","$18,061,660.37"
Northeast,US,North America,"$2,402,176.85","$7,820,209.63"
Central,US,North America,"$3,072,175.12","$8,913,299.25"
Southwest,US,North America,"$10,510,853.87","$27,125,275.34"
Southeast,US,North America,"$2,538,667.25","$8,884,099.37"
Canada,CA,North America,"$6,771,829.14","$18,398,929.19"
France,FR,Europe,"$4,772,398.31","$8,119,749.35"
Germany,DE,Europe,"$3,805,202.35","$5,479,819.58"
Australia,AU,Pacific,"$5,977,814.92","$11,814,376.10"
United Kingdom,GB,Europe,"$5,012,905.37","$8,574,048.71"


<h3>Phase 4: Data Analysis</h3>

<h4>Answering the 5 questions</h4>

<p>to answer the 5 questions, I used the proc tabulate step, I used a variety of labels and formats statements
to modify the result of the procedure.</p>

<h5>Q1: What is the Total Due for all the North American Regions?</h5>

<p>$89,203,473.14</p>

In [7]:
* What is the Total Due for all the North American Regions?;
proc tabulate
	data=project.SalesAnalysis;
	class Group;
	var Subtotal;
	table Subtotal=' '*f=dollar16.2, Group='All Regions' / rts=25 row=float;
	keylabel sum='Total Due';
run;
proc tabulate
	data=project.SalesAnalysis;
	class Group;
	var Subtotal;
	table Subtotal=' '*f=dollar16.2, Group='Only North America Regions' / rts=25 row=float;
	keylabel sum='Total Due';
	where Group='North America';
run;

<h5>Q2: What is the total Sales YTD for U.S.?</h5>
<p>$26,411,059.88</p>

In [8]:
* What is the total Sales YTD for U.S.?;
proc tabulate
	data=project.SalesAnalysis;
	class CountryRegionCode;
	var SalesYTD;
	table SalesYTD=' '*f=dollar16.2, CountryRegionCode='By Country Region Code' / rts=25 row=float;
	keylabel sum='Sales YTD';
run;
proc tabulate
	data=project.SalesAnalysis;
	class CountryRegionCode;
	var SalesYTD;
	table SalesYTD=' '*f=dollar16.2, CountryRegionCode='By Country Region Code' / rts=25 row=float;
	keylabel sum='Sales YTD';
	where CountryRegionCode='US';
run;

<h5>Q3: How much is due from France and Germany?</h5>
<p>$8,119,749.35 and $5,479,819.58</p>

In [9]:
* How much is due from France and Germany?;
proc tabulate
	data=project.SalesAnalysis;
	class Name;
	var Subtotal;
	table Subtotal=' '*f=dollar16.2, Name='Total Amount Due' / rts=25 row=float;
	keylabel sum='Total Due';
	where Name in('France','Germany');
run;

<h5>Q4: What is the total Sales YTD for Europe?</h5>
<p>$13,590,506.02</p>

In [10]:
* What is the total Sales YTD for Europe?;	
proc tabulate
	data=project.SalesAnalysis;
	class Group;
	var SalesYTD;
	table SalesYTD=' '*f=dollar16.2, Group='Total Sales YTD' / rts=25 row=float;
	keylabel sum='Total Due';
run;	
proc tabulate
	data=project.SalesAnalysis;
	class Group;
	var SalesYTD;
	table SalesYTD=' '*f=dollar16.2, Group='Total Sales YTD In Europe' / rts=25 row=float;
	keylabel sum='Total Due';
	where Group='Europe';
run;

<h5>Q5: How many total territories in U.S?</h5>
<p>12,041</p>

<p>In this question, I had to go back to the SalesDetails dataset and then formatted all territories into
one value so the tabulate step would count the same value for each country.</p>

In [11]:
* How many total territories in U.S?;
proc format;
	value Territoryfmt
	low-high='Territories';
run;
proc tabulate
	data=project.SalesDetails;
	format TerritoryID Territoryfmt.;
	class CountryRegionCode TerritoryID;
	table TerritoryID=' ', CountryRegionCode='Country Code' / rts=25 row=float;
	keylabel n=' ';
run;
proc tabulate
	data=project.SalesDetails;
	format TerritoryID Territoryfmt.;
	class CountryRegionCode TerritoryID;
	table TerritoryID=' ', CountryRegionCode='Country Code' / rts=25 row=float;
	keylabel n=' ';
	where CountryRegionCode='US';
run;

<h5>Create at least one chart</h5>
<p>I used the proc gchart to create a bar chart and a pie chart, I wanted to show the total amount due
for each country side by side so it was easy to compare but after making the first bar graph and
adding all US regions together I felt the need to show the amount due for each region of the US
and thus I used the pie chart.</p>

In [12]:
* Chart;
proc format;
	value 	$CountryRegionName
			'US'='United States'
			'CA'='Canada'
			'AU'='Australia'
			'GB'='United Kingdom'
			'FR'='France'
			'DE'='Germany';
run;
title'Total Due BY Country';
proc gchart
	data=project.SalesDetails;
	format CountryRegionCode $CountryRegionName.;
	label  CountryRegionCode='Country'
			TotalDue="Total Due in USD";
	vbar CountryRegionCode/ sumvar=TotalDue descending noframe;
run;
title'Total Due & Percentages in The US by Region';
proc gchart
	data=project.SalesDetails;
	pie Name/ sumvar=TotalDue descending clockwise angle=90 percent=inside noheading;
	where CountryRegionCode='US';
run;
quit;